In [15]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_experiments.library.tomography import ProcessTomography
from qiskit import QuantumCircuit
from qiskit_ibm_runtime.fake_provider import FakeKyoto
from qiskit_aer import AerSimulator
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit.quantum_info import Chi, Choi, Kraus, SuperOp
import numpy as np
from qiskit_experiments.framework import ParallelExperiment, BatchExperiment
import gen_funcs

In [16]:
service = QiskitRuntimeService(channel="ibm_quantum")
# backend = service.backend("ibm_kyoto")
backend = service.least_busy(operational=True, simulator=False)

In [17]:
# simulator = AerSimulator.from_backend(FakeKyoto())
# ns_simulator = AerSimulator.from_backend(GenericBackendV2(num_qubits=2))

In [18]:
base_circ = QuantumCircuit(1)
base_circ.id(0)
base_circ.repeat(2)

In [19]:
circ_ls = gen_funcs.gen_circ_ls(base_circ, 3)
qubit_ls = gen_funcs.gen_qubit_ls(6, 2, repeat=False)

In [22]:
parallel_qpt = gen_funcs.parallel_exp_1q2q(circ_ls, backend, qubit_ls)

AttributeError: module 'gen_funcs' has no attribute 'parallel_exp_1q2q'